<a href="https://colab.research.google.com/github/nankivel/capstone/blob/main/ResNet7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.5 MB/s eta 0:00:00


In [11]:
import os
import datetime
import chess
import chess.engine
import random
import numpy as np
import pydot
from tqdm import tqdm
import io
import json
import graphviz
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# help functions

# def stockfish(board, depth):
#   with chess.engine.SimpleEngine.popen_uci(r"C:\Users\Richard.Huang\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe") as sf:
#     result = sf.analyse(board, chess.engine.Limit(depth=depth))
#     score = result['score'].white().score()
#   return score

def board_encoder(board):
  encoded_board = np.zeros([8,8,15]).astype(np.int8)
  fen = board.fen()
  fen_field = fen.split(' ')
  PiecePlacement = fen_field[0].split('/')
  piece_dict = {"R":0, "N":1, "B":2, "Q":3, "K":4, "P":5,
                "r":6, "n":7, "b":8, "q":9, "k":10, "p":11
                }
  for rank in range(8):
    pieces = ''
    for c in PiecePlacement[rank]:
      if c.isnumeric():
        pieces += '-'*int(c)
      else:
        pieces += c
    for file in range(8):
      if pieces[file] != '-':
        encoded_board[rank, file, piece_dict[pieces[file]]] = 1
  # plane 12 encodes all the legal moves of white
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
    encoded_board[7-np.unravel_index(move.to_square, (8,8))[0], np.unravel_index(move.to_square, (8,8))[1], 12] = 1
  # plane 13 encodes all the legal moves of black
  board.turn = chess.BLACK
  for move in board.legal_moves:
    encoded_board[7-np.unravel_index(move.to_square, (8,8))[0], np.unravel_index(move.to_square, (8,8))[1], 13] = 1
  board.turn = aux
  # plane 14 encodes the current player to move: white is 1, black is 0
  if fen_field[1] == 'w':
    encoded_board[:,:,14] = 1
  else:
    encoded_board[:,:,14] = 0
  return encoded_board

In [15]:
# load training data

# inputdata1 = np.load("/content/drive/MyDrive/dataset_lichess1.Mar18.npz")
# inputdata2 = np.load("/content/drive/MyDrive/dataset_lichess2.Mar18.npz")
# inputdata3 = np.load("/content/drive/MyDrive/dataset_lichess3.Mar18.npz")

# X = np.concatenate([inputdata1['X'], inputdata2['X'], inputdata2['X']])
# y = np.concatenate([inputdata1['y'], inputdata2['y'], inputdata2['y']])

#np.savez("/content/drive/MyDrive/dataset_lichess123.Mar18.npz", X=X, y=y)

inputdata = np.load("/content/drive/MyDrive/dataset_lichess123.Mar18.npz")

X = inputdata['X']
y = inputdata['y']
X.shape

(298594, 8, 8, 15)

In [7]:
# model components

class ConvBlock(nn.Module):
    def __init__(self, channel_in=15, channel_out=256, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        self.channel_in = channel_in
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(channel_out)

    def forward(self, s):
        if s.dtype != torch.float32:
            s = s.float()
        s = s.view(-1, self.channel_in, 8, 8)  # batch_size x channels x board_x x board_y
        s = self.conv1(s)
        s = self.bn1(s)
        s = F.relu(s)
        return s

class ResBlock(nn.Module):
    def __init__(self, channel_in=256, channel_out=256, kernel_size=3, stride=1,
                 downsample=None):
        super(ResBlock, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_out)
        self.conv2 = nn.Conv2d(channel_out, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_out)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = F.relu(out)
        return out

class OutBlock(nn.Module):
    def __init__(self):
        super(OutBlock, self).__init__()
        self.conv = nn.Conv2d(256, 1, kernel_size=1, stride=1)
        self.bn = nn.BatchNorm2d(1)
        self.fc1 = nn.Linear(8*8, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self,s):
        v = self.conv(s)
        v = self.bn(v)
        v = F.relu(v)
        v = v.view(-1, 8*8)  # batch_size x channels x board_x x board_y
        v = self.fc1(v)
        v = F.relu(v)
        v = self.fc2(v)
        v = torch.tanh(v)

        return v

In [8]:
# 7 layers of residual blocks

class ResNet7(nn.Module):
    def __init__(self):
        super(ResNet7, self).__init__()
        self.conv = ConvBlock()
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(7)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

In [19]:
class training_data():
    def __init__(self, dataset): # dataset = np.array of (s, v)
        # self.X = dataset[:,0]
        # self.y = dataset[:,1]
        self.X = dataset['X']
        self.y = np.array(dataset['y'] / abs(dataset['y']).max(),
                          dtype=np.float32) # normalized to [-1,1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self,idx):
        return self.X[idx].transpose(2,0,1), self.y[idx]

def train(net, dataset, batch_size=100, epoch_start=0, epoch_stop=20):
    net.train()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                               milestones=[100,200,300,400],
                                               gamma=0.2)
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.1, verbose=True)
    train_set = training_data(dataset)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                              num_workers=2, pin_memory=False)
    losses_per_epoch = []
    for epoch in range(epoch_start, epoch_stop):
        total_loss = 0.0
        losses_per_batch = []
        for i,data in enumerate(train_loader):
            state, value = data
            if cuda:
                state, value = state.cuda().float(), value.cuda().float()
            value = value.float()  # Convert to torch.float32
            optimizer.zero_grad()
            value_pred = net(state) # value_pred = torch.Size([batch, 1])
            loss = criterion(value_pred[:,0], value)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches of size = batch_size
                print('Process ID: %d [Epoch: %d, %5d/ %d points] total loss per batch: %.3f' %
                      (os.getpid(), epoch + 1, (i + 1)*batch_size, len(train_set), total_loss/10))
                print("Value:",value[0].item(),value_pred[0,0].item())
                losses_per_batch.append(total_loss/10)
                total_loss = 0.0
        losses_per_epoch.append(sum(losses_per_batch)/len(losses_per_batch))
        scheduler.step()
        # if len(losses_per_epoch) > 100:
        #     if abs(sum(losses_per_epoch[-4:-1])/3-sum(losses_per_epoch[-16:-13])/3) <= 0.01:
        #         break
    fig = plt.figure()
    ax = fig.add_subplot(222)
    ax.scatter([e for e in range(1,epoch_stop+1,1)], losses_per_epoch)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss per batch")
    ax.set_title("Loss vs Epoch")
    print('Finished Training')
    plt.savefig("/content/drive/MyDrive/Loss_vs_Epoch_%s.png" % datetime.datetime.today().strftime("%Y-%m-%d"))

In [ ]:
# train net
net = ResNet7()
cuda = torch.cuda.is_available()
if cuda:
  net.cuda()
# current_net_filename = r"C:\Users\Richard.Huang\OneDrive - TGS\Desktop\Jupyter\AlphaZero\model_data\ResNet7_training1.pth.tar"
# checkpoint = torch.load(current_net_filename)
# net.load_state_dict(checkpoint['state_dict'])
train(net,inputdata, epoch_stop=100)
# save results
torch.save({'state_dict': net.state_dict()}, "/content/drive/MyDrive/ResNet7_trained1.pth.tar")

Process ID: 647 [Epoch: 1,  1000/ 298594 points] total loss per batch: 0.179
Value: -0.013220339082181454 -0.009521770291030407
Process ID: 647 [Epoch: 1,  2000/ 298594 points] total loss per batch: 0.003
Value: -0.028248587623238564 0.025194507092237473
Process ID: 647 [Epoch: 1,  3000/ 298594 points] total loss per batch: 0.002
Value: -0.001129943528212607 0.01441993284970522
Process ID: 647 [Epoch: 1,  4000/ 298594 points] total loss per batch: 0.001
Value: 0.00022598869691137224 0.0014093155041337013
Process ID: 647 [Epoch: 1,  5000/ 298594 points] total loss per batch: 0.001
Value: -0.0014689265517517924 0.013506503775715828
Process ID: 647 [Epoch: 1,  6000/ 298594 points] total loss per batch: 0.001
Value: 0.008926553651690483 -0.001572847948409617
Process ID: 647 [Epoch: 1,  7000/ 298594 points] total loss per batch: 0.002
Value: 0.032316382974386215 0.012077235616743565
Process ID: 647 [Epoch: 1,  8000/ 298594 points] total loss per batch: 0.002
Value: 0.14146892726421356 8.618